In [7]:
import pandas as pd
import re

# NLTK: A basic, popular NLP package. Find many examples of applications at https://www.nltk.org/book/
# Install guide: https://www.nltk.org/install.html
import nltk
nltk.download('punkt') # you will probably need to do this
nltk.download('wordnet') # and this
nltk.download('stopwords') # aand this
nltk.download('sentiwordnet')
porter = nltk.PorterStemmer()
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("danish") 
stop_words_list = nltk.corpus.stopwords.words('danish')

#lemitizer
import lemmy
# Create an instance of the standalone lemmatizer.
lemmatizer = lemmy.load("da")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jonat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jonat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jonat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\Jonat\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


In [8]:
data=pd.read_csv("data2.csv") # read the initial dataframe

In [9]:
def preprocess(text):
    remowe_html = re.sub(r'<.*?>','', text) # remowes any links
    punct_removed_2 = re.sub(r'[^\w\s]','',remowe_html) # remows punctuation 
    sent = [i for i in nltk.tokenize.word_tokenize(punct_removed_2) if i not in stop_words_list]# tokeniser the words, and remowes the stop words
    liste=[]
    for i in sent: # only stem words with that begins with lowercase, thus hoping to not stem names
        if i[0].isupper()==False:
            x=stemmer.stem(i)
        else:
            x=i
        liste.append(x)
    final = lemmatizer.lemmatize("v", ' '.join(liste))
    #final=[lemmatizer.lemmatize("v", i) for i in sent] # lematise the words, using danish lematiser
    numbers = re.sub(r'\d+', 'Unikttal' , final[0])
    #numbers=[re.sub(r'\d+', 'Unikttal' ,i[0]) for i in final] # findding all numbers, and returning a unique number
    ret = numbers.lower()
    #ret=[i.lower() for i in numbers] # lowercasse evertyning to make less words
  
    return ret # returns the propreocces words


In [10]:
df1=data.copy() # makes a copy of the dataframe
df1

,url,comments,title,subtitle,body,date
0,https://ekstrabladet.dk/nyheder/politik/danskp...,1952,Statsministeren åbner for muligt påbud om mund...,NaN,"Statsminister Mette Frederiksen (S) åbner for,...",7. aug. 2020
1,https://ekstrabladet.dk/nyheder/politik/danskp...,448,Mette F. afviser at åbne for nattelivet i august,"Mette Frederiksen (S) siger, at hun ikke kan s...","Statsminister Mette Frederiksen (S) siger, at ...",7. aug. 2020
2,https://ekstrabladet.dk/flash/filmogtv/daarlig...,34,Dårligt nyt til 'Venner'-fans,NaN,Der var gode nyheder til fans af serien 'Frien...,7. aug. 2020
3,https://ekstrabladet.dk/nyheder/samfund/flemmi...,227,Flemming dødtræt af Elgiganten: Gør det aldrig!,NaN,"Per, Mikkel, Jeppe, Allan, Zanshi og Jasmin. F...",7. aug. 2020
4,https://ekstrabladet.dk/nyheder/samfund/venstr...,27,"Venstre vil have håndtrykket tilbage, hvis det...","Virolog mener dog ikke, at tidspunktet er det ...","Hvis du synes, det var svært at finde ud af, h...",7. aug. 2020
...,...,...,...,...,...,...
26240,https://ekstrabladet.dk/sport/fodbold/dansk_fo...,11,Halv milliard rammer DBU-kassen i ny tv-aftale,En stor landsholdsaftale for europæisk fodbold...,Dansk Boldspil-Union (DBU) nyder for tiden god...,29. feb. 2020
26241,https://ekstrabladet.dk/nyheder/samfund/bloede...,191,"Bløder: Mister 1,15 milliarder kr.",NaN,Verdens største øl-producent Anheuser-Busch In...,29. feb. 2020
26242,https://ekstrabladet.dk/sport/anden_sport/boks...,7,- En god gammeldags svinestreg,NaN,Se professionel top-boksning i aften. Køb adga...,29. feb. 2020
26243,https://ekstrabladet.dk/musik/intlmusiknyt/bri...,31,Britisk band i stor busulykke,NaN,Torsdag aften måtte 19 personer fragtes til ho...,29. feb. 2020


In [ ]:
# process the data in the dataframe copy
x=len(df1)
list_=[]

for i  in range(x):
    
    df1.dropna(subset = ["title"], inplace=True) # droppes articles with missing in the tile and reset index
    df=df1.reset_index(drop=True)
    
    x=preprocess(df['title'][i]) # preporcces and append to list
    list_.append(x)
    


In [12]:
# save the data
df2=pd.DataFrame(df)
df2['proc_title']=pd.Series(list_) # append the preproces data to a new collum 
df2.to_csv("processed_data2.csv",index=False) #saves it to a csv file 

In [13]:
df_proc = pd.read_csv('processed_data2.csv')

In [14]:
# Filter words that appear less than 10 times
liste=10>df_proc['proc_title'].str.split(expand=True).stack().value_counts()

In [15]:
# make df from the list
df=pd.DataFrame(liste)
df=df.reset_index()
df.columns = ['index', 'true']

In [16]:
# only the words with less than 10
df_False = df[df.true==True]
# create string with all the words we remove
list_drop = df_False['index']
list_drop = list_drop.tolist()
list_drop = ' '.join(list_drop)

In [17]:
# split the dataset, to be able to sort through the words
for i, sent in enumerate(df_proc['proc_title']):
    df_proc['proc_title'][i] = df_proc['proc_title'][i].split(' ')

C:\Users\Jonat\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
# create list with all the stripped titles
word_list_2 = []
for i, sent in enumerate(df_proc['proc_title']):
    word_list = []
    for word in sent:
        if word not in list_drop:
            word_list.append(word)
    word_list_2.append(word_list)

# save to df and drop titles with all words removed
df_proc['proc_title_rm'] = word_list_2
df_proc = df_proc[df_proc.proc_title_rm!='']

In [19]:
# join the titles to one string instead of list of strings
for i, sent in enumerate(df_proc['proc_title_rm']):
    df_proc['proc_title_rm'][i] = ' '.join(df_proc['proc_title_rm'][i])
df_proc = df_proc.reset_index()  # reset index

C:\Users\Jonat\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
df_proc = df_proc[df_proc.proc_title_rm!=''] # final drop of empty titles

In [21]:
df_proc.to_csv("no_words2.csv",index=False) #saves it to a csv file 

In [22]:
df_proc # check df

,index,url,comments,title,subtitle,body,date,proc_title,proc_title_rm
0,0,https://ekstrabladet.dk/nyheder/politik/danskp...,1952,Statsministeren åbner for muligt påbud om mund...,NaN,"Statsminister Mette Frederiksen (S) åbner for,...",7. aug. 2020,"[statsministeren, åbn, mul, påbud, mundbind]",statsministeren
1,1,https://ekstrabladet.dk/nyheder/politik/danskp...,448,Mette F. afviser at åbne for nattelivet i august,"Mette Frederiksen (S) siger, at hun ikke kan s...","Statsminister Mette Frederiksen (S) siger, at ...",7. aug. 2020,"[mette, f, afvis, åbn, natteliv, august]",august
2,2,https://ekstrabladet.dk/flash/filmogtv/daarlig...,34,Dårligt nyt til 'Venner'-fans,NaN,Der var gode nyheder til fans af serien 'Frien...,7. aug. 2020,"[dårligt, nyt, vennerfans]",dårligt
5,5,https://ekstrabladet.dk/sport/fodbold/udenland...,24,Bizar opladning: Derfor smører han sig ind i b...,NaN,"Det er næppe nogen hemmelighed, at Wolverhampt...",7. aug. 2020,"[bizar, opladning, derfor, smør, babyoli]",derfor
9,9,https://ekstrabladet.dk/sport/formel_1/aarets-...,7,"Årets bedste for Lundgaard: - Undskyld, gutter",NaN,En overgang lignede det en magtdemonstration f...,7. aug. 2020,"[årets, bedst, lundgaard, undskyld, gut]",årets
...,...,...,...,...,...,...,...,...,...
26153,26153,https://ekstrabladet.dk/sport/anden_sport/boks...,14,Vildt knockout: Gik ud som et lys,NaN,Landry Kore (nu 11-0-1) hev en vild knockout f...,29. feb. 2020,"[vildt, knockout, gik, lys]",vildt
26165,26165,https://ekstrabladet.dk/sport/formel_1/f1-sens...,15,F1-sensation lurer: Vanvittigt spændende,NaN,MONTMELÓ (Ekstra Bladet): Kan Racing Point vir...,29. feb. 2020,"[funikttalsensation, lur, vanvittigt, spænd]",vanvittigt
26166,26166,https://ekstrabladet.dk/nyheder/politik/boris-...,14,Boris Johnson skal giftes og have endnu et bar...,"55-årige Boris Johnson, der har fire voksne bø...","Den britiske premierminister, 55-årige Boris J...",29. feb. 2020,"[boris, johnson, gift, endnu, barn, som]",boris endnu
26167,26167,https://ekstrabladet.dk/kup/22.000-kr.-brugt-u...,203,22.000 kr. brugt under pub crawl: Kræver Norde...,NaN,"Mange kender de økonomiske tømmermænd, der mel...",29. feb. 2020,"[unikttal, kr, brugt, pub, crawl, kræver, nord...",kræver
